# Libraries

In [1]:
import pandas as pd
import json
from enum import Enum
import datetime

# Common functions

## Cast functions

In [2]:
def cast_col_to_datetime(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    df[column_name] = pd.to_datetime(df[column_name], format='ISO8601')
    return df

def cast_col_to_str(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    df[column_name] = df[column_name].astype(str)
    return df

def cast_col_to_int(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    df[column_name] = df[column_name].astype(int)
    return df

def cast_col_to_date(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    df[column_name] = pd.to_datetime(df[column_name], format='%d.%m.%Y')
    return df

class ColCast(Enum):
    TO_DATETIME = cast_col_to_datetime
    TO_DATE = cast_col_to_date
    TO_STRING = cast_col_to_str
    TO_INTEGER = cast_col_to_int

def cast_columns(df: pd.DataFrame, cast_mapping: dict[str:ColCast]) -> pd.DataFrame:
    # perform check if provided mapping matches dataframe
    columns_match = set(df.columns) == set(cast_mapping.keys())
    if not columns_match:
        raise Exception("Provided mapping do not match the columns of the dataframe.")
    
    for col_name, cast_func in cast_mapping.items():
        df = cast_func(df, col_name)

    return df

# Extract

## Read data.csv

In [3]:
applications_df = pd.read_csv("./data.csv")

# Transform

## Apply datatype casting on dataframe columns

In [4]:
applications_datatypes = {
    "id": ColCast.TO_INTEGER,
    "application_date": ColCast.TO_DATETIME,
    "contracts": ColCast.TO_STRING, 
}

In [5]:
casted_df = cast_columns(applications_df.copy(), applications_datatypes)

## Parse contracts column

In [6]:
casted_df['contracts'] = casted_df['contracts'].replace('nan', '[{}]')
casted_df['contracts'] = casted_df['contracts'].apply(json.loads)
exploded_df = casted_df.explode('contracts').reset_index(drop=True)

In [7]:
flattened_df = pd.json_normalize(exploded_df['contracts'])
parsed_df = pd.concat([exploded_df, flattened_df], axis=1)

## Calculate tot_claim_cnt_l180d feature

In [8]:
current_date = pd.Timestamp(datetime.datetime.now())
minus_180_days = current_date - pd.Timedelta(days=180)

In [9]:
claim_df = parsed_df.copy()
claim_df['claim_date'] = pd.to_datetime(parsed_df['claim_date'])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_30636\72844443.py:2: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  claim_df['claim_date'] = pd.to_datetime(parsed_df['claim_date'])


In [10]:
claim_df = claim_df[(claim_df['claim_date'].notnull()) & (claim_df['claim_date'] > minus_180_days)]
cnt_l180d_feature_df = claim_df.groupby('id')['claim_id'].count().reset_index(name='tot_claim_cnt_l180d')

In [11]:
parsed_df = parsed_df.merge(cnt_l180d_feature_df, on='id', how='left')
parsed_df['tot_claim_cnt_l180d'].fillna(-3, inplace=True)
parsed_df['tot_claim_cnt_l180d'] = parsed_df['tot_claim_cnt_l180d'].astype(int)

In [12]:
parsed_df

,id,application_date,contracts,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date,tot_claim_cnt_l180d
0,2925210,2024-02-12 19:22:46.652000+00:00,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3
1,2925211,2024-02-12 19:24:29.135000+00:00,"{'contract_id': 522530, 'bank': '003', 'summa'...",522530,003,500000000,0,13.02.2020,609965,17.02.2020,-3
2,2925211,2024-02-12 19:24:29.135000+00:00,"{'contract_id': '', 'bank': '014', 'summa': ''...",,014,,,28.08.2020,F00013731,,-3
3,2925211,2024-02-12 19:24:29.135000+00:00,"{'contract_id': '', 'bank': '014', 'summa': ''...",,014,,,08.10.2020,F00021301,,-3
4,2925211,2024-02-12 19:24:29.135000+00:00,"{'contract_id': '', 'bank': '014', 'summa': ''...",,014,,,25.11.2020,F00037907,,-3
...,...,...,...,...,...,...,...,...,...,...,...
10064,2926205,2024-02-13 06:09:54.210000+00:00,"{'contract_id': '', 'bank': '014', 'summa': ''...",,014,,,31.05.2023,F00895453,,-3
10065,2926206,2024-02-13 06:09:54.306000+00:00,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3
10066,2926207,2024-02-13 06:09:55.661000+00:00,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3
10067,2926208,2024-02-13 06:09:57.024000+00:00,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3


## Calculate disb_bank_loan_wo_tbc feature

The task `Sum of exposue of loans without TBC loans. Exposure means here field "loan_summa".` is not clear. What does mean `without TBC loans`? Which bank is it?

Feature disb_bank_loan_wo_tbc will be calculated as sum of 'loan_summa' where it is not null, 'contract_date' is not null and TBC Bank will have 014 value.

In [13]:
loan_df = parsed_df[['id', 'bank', 'contract_date', 'loan_summa']]
loan_df = loan_df[(~loan_df['bank'].isin(['LIZ', 'LOM', 'MKO', 'SUG', '014'])) & (loan_df['bank'].notnull()) & (loan_df['contract_date'].notnull()) & (loan_df['contract_date'] != '')]
loan_df = loan_df[(loan_df['loan_summa'] != '')]
loan_df['loan_summa'] = loan_df['loan_summa'].astype('Int64')
loan_wo_tbc_feature_df = loan_df.groupby('id')['loan_summa'].sum().reset_index(name='disb_bank_loan_wo_tbc')


In [14]:
parsed_df = parsed_df.merge(loan_wo_tbc_feature_df, on='id', how='left')
parsed_df['disb_bank_loan_wo_tbc'].fillna(-3, inplace=True)
parsed_df.loc[parsed_df['disb_bank_loan_wo_tbc'] == 0, 'disb_bank_loan_wo_tbc'] = -1

In [15]:
parsed_df

,id,application_date,contracts,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc
0,2925210,2024-02-12 19:22:46.652000+00:00,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3,-3
1,2925211,2024-02-12 19:24:29.135000+00:00,"{'contract_id': 522530, 'bank': '003', 'summa'...",522530,003,500000000,0,13.02.2020,609965,17.02.2020,-3,-1
2,2925211,2024-02-12 19:24:29.135000+00:00,"{'contract_id': '', 'bank': '014', 'summa': ''...",,014,,,28.08.2020,F00013731,,-3,-1
3,2925211,2024-02-12 19:24:29.135000+00:00,"{'contract_id': '', 'bank': '014', 'summa': ''...",,014,,,08.10.2020,F00021301,,-3,-1
4,2925211,2024-02-12 19:24:29.135000+00:00,"{'contract_id': '', 'bank': '014', 'summa': ''...",,014,,,25.11.2020,F00037907,,-3,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
10064,2926205,2024-02-13 06:09:54.210000+00:00,"{'contract_id': '', 'bank': '014', 'summa': ''...",,014,,,31.05.2023,F00895453,,-3,217320883
10065,2926206,2024-02-13 06:09:54.306000+00:00,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3,-3
10066,2926207,2024-02-13 06:09:55.661000+00:00,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3,-3
10067,2926208,2024-02-13 06:09:57.024000+00:00,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3,-3


## Calculate day_sinlastloan feature

In [16]:
last_loan_df = parsed_df[['id', 'summa', 'contract_date', 'application_date']]
last_loan_df = last_loan_df.dropna(subset=['contract_date'])
last_loan_df = last_loan_df[(last_loan_df['summa'] != '') | (last_loan_df['contract_date'] != '')]
last_loan_df['contract_date'] = pd.to_datetime(last_loan_df['contract_date']).dt.date
last_loan_df['application_date'] = pd.to_datetime(last_loan_df['application_date']).dt.date

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_30636\2349048873.py:4: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  last_loan_df['contract_date'] = pd.to_datetime(last_loan_df['contract_date']).dt.date


In [17]:
# custom grouping function
def calculate_days_since_last_loan(group: pd.DataFrame):
    group = group.sort_values('contract_date')
    group['day_sinlastloan'] = group.apply(
        lambda row: (row['application_date'] - group[group['contract_date'] <= row['application_date']]['contract_date'].max()).days, axis=1
    )
    return group

In [18]:
day_sinlastloan_feature_df = last_loan_df.groupby('id').apply(calculate_days_since_last_loan).reset_index(drop=True)
day_sinlastloan_feature_df = day_sinlastloan_feature_df[['id', 'day_sinlastloan']]

In [19]:
parsed_df = parsed_df.merge(day_sinlastloan_feature_df, on='id', how='left')
parsed_df['day_sinlastloan'].fillna(-3, inplace=True)
# Assumed no loans except TBC
parsed_df.loc[parsed_df['disb_bank_loan_wo_tbc'] == -1, 'day_sinlastloan'] = -1
parsed_df['day_sinlastloan'] = parsed_df['day_sinlastloan'].astype(int)

In [20]:
parsed_df = parsed_df[['id', 'tot_claim_cnt_l180d', 'disb_bank_loan_wo_tbc', 'day_sinlastloan']]
parsed_df = parsed_df.drop_duplicates(subset=['id'])
final_df = casted_df.merge(parsed_df, on=['id'], how='inner')

In [21]:
final_df

,id,application_date,contracts,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc,day_sinlastloan
0,2925210,2024-02-12 19:22:46.652000+00:00,[{}],-3,-3,-3
1,2925211,2024-02-12 19:24:29.135000+00:00,"[{'contract_id': 522530, 'bank': '003', 'summa...",-3,-1,-1
2,2925212,2024-02-12 19:24:41.493000+00:00,[{}],-3,-3,-3
3,2925213,2024-02-12 19:24:29.135000+00:00,"[{'contract_id': 522530, 'bank': '003', 'summa...",-3,-1,-1
4,2925214,2024-02-12 19:24:56.857000+00:00,[{}],-3,-3,-3
...,...,...,...,...,...,...
995,2926205,2024-02-13 06:09:54.210000+00:00,"[{'contract_id': 18410, 'bank': '004', 'summa'...",-3,217320883,729
996,2926206,2024-02-13 06:09:54.306000+00:00,[{}],-3,-3,-3
997,2926207,2024-02-13 06:09:55.661000+00:00,[{}],-3,-3,-3
998,2926208,2024-02-13 06:09:57.024000+00:00,[{}],-3,-3,-3


In [22]:
final_df.to_csv('contract_features.csv', index=False)